# Clipping plane visualization of tent slab solution

We show how to convert a tent slab based on a 2D mesh into a 3D mesh for which we can use the webgui's clipping feature.

In [3]:
from netgen.geom2d import unit_square
from ngsolve import Mesh, CoefficientFunction, sqrt, x, y, exp, L2, H1, GridFunction, TaskManager
from ngstents import TentSlab
from ngstents.conslaw import Euler
from ngstents.utils import SlabConverter
from ngsolve.webgui import Draw

In [8]:
maxh = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))

dt = 0.05
tend = 0.25

# using causality constant
local_ctau = True
global_ctau = 2/3
wavespeed = 6
ts = TentSlab(mesh, method="edge")
ts.SetMaxWavespeed(wavespeed)
ts.PitchTents(dt=dt, local_ct=local_ctau, global_ct=global_ctau)
print("max slope", ts.MaxSlope())
print("n tents", ts.GetNTents())

order = 4
V = L2(mesh, order=order, dim=mesh.dim+2)
u = GridFunction(V,"u")
cl = Euler(u, ts, reflect=mesh.Boundaries("left|bottom|right|top"))
cl.SetTentSolver("SARK",substeps=2*order)

d = 5
rho = CoefficientFunction(0.1+exp(-200*((x-0.5)*(x-0.5)+(y-0.5)*(y-0.5))))
m = CoefficientFunction((0,0))
p = CoefficientFunction(0.1+exp(-200*((x-0.5)*(x-0.5)+(y-0.5)*(y-0.5))))
T = 2*p/rho
E = d/4*T*rho + 1/(2*rho)*m*m

cf = CoefficientFunction((rho,m,E))
cl.SetInitial(cf)
# visoptions.scalfunction = "u:1"
# visoptions.vecfunction = None

scene = Draw(u)

# SlabConverter can be used to convert a tent slab based on a 2D spatial mesh to a 3D mesh.
# order_3d must be 1 or 2
order_3d = 2
sc = SlabConverter(ts, p_hd=order_3d)
sc.Convert(tscale=5)
mesh3d = sc.mesh
gfixmap = sc.gfixmap

# Due to memory concerns, we currently store only interpolated vertex dofs
# (and optionally edge dofs if order_3d=2) for the 3D space.
# We define an H1 order 1 or 2 space and associated GridFunction here.
V3d = H1(sc.mesh, order=order_3d)
u3d = GridFunction(V3d, "u3d")

# Set the nested list which gives 3D mesh vertex number
# for front and spatial vertex number
cl.SetIdx3d(gfixmap)

tend = 10*dt

# Define the time steps for which we want to examine the solution using clipping
vecs = {1: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}

t = 0
n = 1
with TaskManager():
    while t < tend - dt/2:
        if n in vecs:
            # Tell Propagate to update the provided vector with the average
            # value for each tent vertex after propagating on the tent.
            u3d.vec.data = vecs[n]
            cl.Propagate(hdgf=u3d)  # Propagate updates u3d 
            vecs[n].data = u3d.vec    # store updated solution vector
        else:
            cl.Propagate()
        t += dt
        n += 1
        print("{:.3f}".format(t))
        scene.Redraw()

max slope 0.11111111111111134
n tents 5502


WebGuiWidget(value={'ngsolve_version': '6.2.2102-124-gbb6f627c8', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

add vertices 0.16024
add volume elements 0.34200
add surface elements 0.25136
make ngsolve mesh 0.03660
make index map 0.6478006839752197
6013 verts, 30273 vol elems, 4824 surf elems in 1.43866.
0.050
0.100
0.150
0.200
0.250
0.300
0.350
0.400
0.450
0.500


## Now we can also visualize the solution on a 3D mesh using clipping

In [9]:
u3d.vec.data = vecs[1]
clip = {'vec': [0, 0, -1], 'dist': 0.01} 
Draw(u3d, clipping=clip)

WebGuiWidget(value={'ngsolve_version': '6.2.2102-124-gbb6f627c8', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

In [10]:
u3d.vec.data = vecs[10]
clip = {'vec': [0, 0, -1], 'dist': 0.01} 
Draw(u3d, clipping=clip)


WebGuiWidget(value={'ngsolve_version': '6.2.2102-124-gbb6f627c8', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

## Observations
We can visualize the solution at an arbitrary time, but the H1 order 1 approximation is not very nice compared to the L2 order 4 solution.

Try changing the H1 order to 2 to see an improvement!